#Assignment 1
###Mark Swanson
###COP 58959 - Summer 2015
###Dr. Shankar
###2015-05-20

Analyze the data at http://catalog.data.gov/dataset/data-gov-visitor-metrics, pretty printing a table and plotting a barchart to show the top ten organizations by total number of views.

Import the necessary modules

In [22]:
import csv, sqlite3
import matplotlib.pyplot as plt

Open and read the CSV file in order to create an array from the data. Create a database, then insert the array of data into the database.

In [23]:
with open('datagovdatasetsviewmetrics.csv', 'rb') as vmcsv:
    vmcsv_reader = csv.DictReader(vmcsv)
    vmfields = vmcsv_reader.fieldnames
    to_db = [(unicode(field[vmfields[0]], "utf8"), unicode(field[vmfields[1]], "utf8"), unicode(field[vmfields[2]], "utf8"), unicode(field[vmfields[3]], "utf8"), unicode(field[vmfields[4]], "utf8")) for field in vmcsv_reader]

con = sqlite3.connect(":memory:")
cur = con.cursor()
cur.execute('DROP TABLE IF EXISTS vmtable;')
cur.execute('CREATE TABLE vmtable(id TEXT, dataset TEXT, org TEXT, views INTEGER, date TEXT);')
cur.executemany('INSERT INTO vmtable (id, dataset, org, views, date) VALUES (?, ?, ?, ?, ?);', to_db)
con.commit()

Select the longest organization title to allow proper formatting of the final table. Select the top 10 organizations based on total view count.

In [24]:
max_org_length = cur.execute('SELECT org_length FROM ( SELECT LENGTH(org) AS org_length, SUM(views) AS sum_views FROM vmtable GROUP BY id ORDER BY sum_views DESC LIMIT 10) as top10views ORDER BY org_length DESC LIMIT 1;').fetchone()[0]
cur.execute('SELECT org, SUM(views) AS sum_views FROM vmtable GROUP BY org ORDER BY sum_views DESC LIMIT 10;')
top_orgs = cur.fetchall()
con.close()

Print a list of the organizations. Insert extra spaces after shorter organization titles.

In [25]:
org_names = []
org_views = []
for row in top_orgs:
    org_names.append(row[0][:13]+'.\n'+row[0].split()[-1])
    org_views.append(row[1])
    print "{:<{}}  {}".format(row[0], max_org_length, row[1])

US Census Bureau, Department of Commerce                                 178510
National Oceanic and Atmospheric Administration, Department of Commerce  155713
U.S. Geological Survey, Department of the Interior                       109848
U.S. Department of Health & Human Services                               93297
Department of Agriculture                                                76948
Department of Defense                                                    45886
Earth Data Analysis Center, University of New Mexico                     41974
Department of Education                                                  39064
Department of Transportation                                             37620
General Services Administration                                          31281


Plot a bar chart of the organizations' total view count.

In [26]:
fig = plt.figure()
ax = fig.add_subplot(111)
index = range(len(org_views))
width = .85

rects1 = ax.bar(range(len(org_views)), org_views, width,
                color='black')

ax.set_xlim(-(1-width),len(index))
ax.set_ylabel('Views')
ax.set_title('Views by Organization')
xTickMarks = org_names
ax.set_xticks(range(len(org_views)))
xtickNames = ax.set_xticklabels(xTickMarks)
plt.setp(xtickNames, rotation=45, fontsize=10)

plt.tight_layout()
plt.show()